# Reglas de asociación

Las reglas de asociación son un método de minería de datos que se utiliza para encontrar relaciones interesantes entre variables en grandes bases de datos. 
Estas reglas tienen la forma *"si A entonces B"*, lo que significa que si ocurre un determinado conjunto de elementos (A), es probable que también ocurra otro conjunto de elementos (B).

Las reglas de asociación son una de las técnicas más utilizadas en la industria minorista, y cuentan con una amplia variedad de aplicaciones, entre las que destacan:
 - El **análisis de la cesta de la compra**: Identificar qué productos se compran con frecuencia juntos en los supermercados. Esto puede ayudar a colocar los productos de forma estratégica para aumentar las ventas. Por ejemplo, una regla de asociación podría ser "si se compra pan, entonces también se compra leche". Uno de los mayores ejemplos del análisis de reglas de asociación es la correlación entre la cerveza y los pañales. Cuando Walmart, una cadena de tiendas en Estados Unidos, estudió el comportamiento de compra de sus clientes, se encontró con que los pañales y las cervezas se compran juntos. La explicación resultó ser que muchos los padres son los encargados de hacer las compras mientras las madres se quedan con el bebé.
 - Algunos **sistemas de recomendación**: Recomendar productos o contenido a los usuarios en función de sus comportamientos o preferencias pasadas. Por ejemplo, en un servicio de streaming de vídeo, si un usuario ha visto muchas películas de ciencia ficción, el sistema puede recomendar otras películas del mismo género. Algoritmos basados en reglas de asociación se usan en empresas como Spotify, Netflix y YouTube.
 - **Diagnóstico médico**: Se pueden usar reglas de asociación para identificar relaciones entre síntomas y enfermedades. Esto puede ayudar a los médicos a realizar diagnósticos más precisos.  

In [1]:
%pip install mlxtend pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Algoritmo Apriori
El algoritmo más popular y clásico, donde los datos se evalúan con reglas de asociación booleanas, es el denominado *Algoritmo a priori*. 
Este algoritmo determina grupos de productos que aparecen frecuentemente, y luego busca relaciones fuertes entre estos y otros productos.

In [2]:
import pandas as pd

df = pd.read_csv('./Datos/datos.csv', names = ['productos'], sep = ',')
df.head()

,productos
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"


Lo primero que debemos hacer es separar cada fila del fichero por las comas, de forma que podamos acceder a los productos individuales comprados por cada cliente.

In [3]:
datos = list(df["productos"].apply(lambda x: x.split(",")))
datos

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

Lo siguiente que tenemos que hacer es generar un dataframe con el mismo numero de columnas para cada compra, y con valores verdadero/falso para cada una de esas columnas. Es decir, necesitamos realizar una codificación OneHot.
 
Para ello usaremos un `TransactionEncoder`. Los productos que los clientes compran en una transacción se representarán con el 1, mientras que los que no se compran aparecerán con un 0. 

In [4]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
datos_codificados = encoder.fit_transform(datos)
df = pd.DataFrame(datos_codificados, columns=encoder.columns_)

df

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,True,True,False,False,False,False,False,False,False,True
6,False,False,False,False,False,True,False,True,False,False,True
7,True,False,True,False,False,False,False,True,False,False,True
8,False,False,True,False,False,False,True,True,False,False,True
9,False,False,True,False,False,False,False,False,True,False,False


Tras estas transformaciones, podemos ver facilmente que, por ejemplo, 7 clientes han comprado galletas, 8 han comprado café, y 5 han comprado leche.

### Generación de itemsets fecuentes con el modelo Apriori

El siguiente paso para la extracción de las reglas de asociación es obtener los itemsets frecuentes. En este caso emplearemos un modelo [Apriori](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/). Este algoritmo basa su funcionamiento en la generación iterativa de candidatos y su podado segun la frecuencia de ese candidato.

Cuando creamos un modelo Apriori debemos especificar el soporte mínimo, es decir, la frecuencia relativa de aparición de un conjunto de elementos en el conjunto de datos de transacciones a partir de la cual consideraremos el itemset frecuente:

$$
Soporte(I)= \frac{\text{Numero de transacciones en las que aparece el itemset }I}{\text{Numero de transacciones total}} 
$$
​
En este ejemplo usaremos un soporte mínimo de 0.2

In [5]:
from mlxtend.frequent_patterns import apriori

df_a = apriori(df, min_support = 0.2, use_colnames = True, verbose = 1)
df_a

Processing 42 combinations | Sampling itemset size 3


,support,itemsets
0,0.35,(BISCUIT)
1,0.20,(BOURNVITA)
2,0.65,(BREAD)
3,0.40,(COFFEE)
4,0.30,(CORNFLAKES)
5,0.25,(MAGGI)
6,0.25,(MILK)
7,0.30,(SUGER)
8,0.35,(TEA)
9,0.20,"(BISCUIT, BREAD)"


### Generación de itemsets fecuentes con el modelo FPGrowth

Una alternativa al modelo Apriori es el [FPGrowth](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/). Este modelo, al contrario que el Apriori, no genera candidatos a patrones frecuentes, si no que va extendiendo un arbol en profundidad para generar los itemsets, lo que lo hace mucho mas eficiente.

Al igual que con el modelo Apriori, debemos especificar un soporte mínimo para los intemsets generados. Si volvemos a usar un soporte mínimo de 0.2:

In [6]:
from mlxtend.frequent_patterns import fpgrowth

df_f = fpgrowth(df, min_support = 0.2, use_colnames = True, verbose = 1)
df_f

9 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (BREAD)
1 itemset(s) from tree conditioned on items (BISCUIT)
1 itemset(s) from tree conditioned on items (MILK)
1 itemset(s) from tree conditioned on items (CORNFLAKES)
1 itemset(s) from tree conditioned on items (TEA)
0 itemset(s) from tree conditioned on items (BOURNVITA)
1 itemset(s) from tree conditioned on items (MAGGI)
0 itemset(s) from tree conditioned on items (COFFEE)
2 itemset(s) from tree conditioned on items (SUGER)
0 itemset(s) from tree conditioned on items (SUGER, COFFEE)
0 itemset(s) from tree conditioned on items (SUGER, BREAD)


,support,itemsets
0,0.65,(BREAD)
1,0.35,(BISCUIT)
2,0.25,(MILK)
3,0.30,(CORNFLAKES)
4,0.35,(TEA)
5,0.20,(BOURNVITA)
6,0.25,(MAGGI)
7,0.40,(COFFEE)
8,0.30,(SUGER)
9,0.20,"(BISCUIT, BREAD)"


### Generación de reglas

Una vez hayados los itemsets frecuentes, podemos proceder a generar las reglas de asociación que relacionan dichos itemsets frecuentes. 

A la hora de generar las reglas, debemos definir un umbral mínimo para las reglas generadas. Este umbral puede definirse en terminos de diferentes métricas:

 - `support`: El soporte mide la frecuencia con la que la regla se aplica al conjunto de datos. Es decir, si la regla es $X \rightarrow Y$, el soporte se calcula como la proporción de transacciones que contienen tanto el antecedente $X$ como el consecuente $Y$.
    $$
        \text{support}(X \rightarrow Y) = \frac{\text{Numero de transacciones en las que aparecen }X\text{ e }Y}{\text{Numero de transacciones total}}
    $$

 - `confidence`: La confianza es una medida de la probabilidad condicional de que ocurra $Y$ dado que ha ocurrido $X$. Es decir, si la regla es $X \rightarrow Y$, la confianza será el porcentaje de veces que el producto $Y$ se compra, dado que se ha comprado el producto $X$.
    $$
        \text{confidence}(X \rightarrow Y) = \frac{\text{support}(X \cup Y)}{\text{support}(X)}
    $$

 - `lift`: El lift de una regla mide qué tan probable es que se compre $Y$ cuando se compra $X$, en comparación con la probabilidad de comprar $Y$ sin considerar la compra de $X$.
    $$
        \text{lift}(X \rightarrow Y) = \frac{\text{confidence}(X \rightarrow Y)}{\text{support}(Y)}
    $$

 - `leverage`: El leverage de una regla mide la diferencia entre la frecuencia observada de que el antecedente y el consecuente aparezcan juntos y la frecuencia esperada si fueran estadísticamente independientes.
    $$
        \text{leverage}(X \rightarrow Y) = \text{support}(X \rightarrow Y) - (\text{support}(X) \times \text{support}(Y))
    $$
 
 - `conviction`: La conviction de una regla de asociación es una métrica que intenta medir la fuerza de la implicación de la regla, considerando la frecuencia con la que el antecedente ocurre sin el consecuente. 
    $$
        \text{conviction}(X \rightarrow Y) = \frac{1 - \text{support}(Y)}{1 - \text{confidence}(X \rightarrow Y)}
    $$

 - `zhangs_metric`: La métrica de Zhang es otra medida utilizada para evaluar el interés de las reglas de asociación teniendo en cuenta la coocurrencia y la coocurrencia de anetcendente y consecuente en el dataset. Fue propuesta por Zhang en el 2000 y busca abordar algunas limitaciones de otras métricas como la confianza y el lift, especialmente en situaciones con datos muy desequilibrados.
 
    $$
        \text{zhangs}(X \rightarrow Y) = \frac{\text{leverage}(X \rightarrow Y)}{\text{max}([\text{support}(X \rightarrow Y) \times (1 - \text{support}(X))], [\text{support}(X) \times (\text{support}(Y) - \text{support}(X \rightarrow Y))])}
    $$

En este ejemplo usaremos un nivel mínimo de confianza del 60%. 

In [7]:
from mlxtend.frequent_patterns import association_rules

df_rules = association_rules(df_a, metric = "confidence", min_threshold = 0.6)
df_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,1.0,0.0375,1.75,0.250000,0.285714,0.428571,0.553846
1,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,1.0,0.0050,1.05,0.035714,0.266667,0.047619,0.487179
2,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,1.0,0.0800,1.80,0.571429,0.400000,0.444444,0.583333
3,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,1.0,0.0800,1.80,0.571429,0.400000,0.444444,0.583333
4,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,1.0,0.1125,3.25,0.750000,0.500000,0.692308,0.685714


<div class="alert alert-info">

**Ejercicio:**

A partir de las reglas obtenidas en el paso anterior:

- ¿Cual es la probabilidad de ver ventas de azúcar? ¿Y de pan?
- ¿Cual es la probabilidad de que se compren juntos pan y azucar?
- ¿Que porcentaje de compradores de azucar compran tambien pan?

</div>

**1. Probabilidad de ver ventas de azúcar y de pan**

In [8]:
prob_azucar = df_a.loc[df_a['itemsets'] == frozenset({'SUGER'}), 'support'].values[0]
prob_pan = df_a.loc[df_a['itemsets'] == frozenset({'BREAD'}), 'support'].values[0]
print(f"P(azúcar) = {prob_azucar:.4f}")
print(f"P(pan) = {prob_pan:.4f}")

P(azúcar) = 0.3000
P(pan) = 0.6500


**2. Probabilidad conjunta de comprar pan y azúcar**

In [9]:
prob_2 = df_a.loc[df_a['itemsets'] == frozenset({'BREAD', 'SUGER'}), 'support'].values[0]
print(f"P(pan y azúcar) = {prob_2:.4f}")

P(pan y azúcar) = 0.2000


**3. ¿Qué porcentaje de compradores de azúcar compran también pan?**

In [10]:
prob_3 = prob_2 / prob_azucar * 100
print(f"P(pan | azúcar) = {prob_3:.2f}%")

P(pan | azúcar) = 66.67%


<div class="alert alert-info">

**Ejercicio:**

El fichero `pedidos.csv` contiene la información sobre 3.000.000 facturas de un supermecado. En el fichero `productos.csv` tenéis la correspondencia entre códigos de producto y el nombre de los mismos.

Si consideramos que un itemset es frecuente cuando aparece en al menos 300 transacciones:
- ¿Qué soporte debemos establecer para calcular los itemsets frecuentes? ¿Cuántos itemsets frecuentes encontramos? ¿Cuál de ellos tiene el soporte más alto?
- ¿Cuántos de los itemsets frecuentes contienen el producto “Seedless Red Grapes”? ¿Cuál de los que tienen más de dos items tiene el soporte más alto?
- ¿Cuántos de los itemset frecuentes son subconjuntos de otro itemset?

Determina el valor necesario de confianza para que se generen entre 150 y 200 reglas con el soporte anteriormente definido. Indica el número de transacciones en las que se verifica la regla con la confianza más alta y que, además, incluyan “Organic Strawberries” en el antecedente o el consecuente.
</div>

In [29]:
pedidos = pd.read_csv("./Datos/pedidos.csv")[:3000000]
productos = pd.read_csv("./Datos/productos.csv", index_col='product_id')
pedidos["product_id"] = pedidos["product_id"].map(productos.to_dict()["product_name"])
grouped = pedidos.groupby('order_id')['product_id'].apply(list)
encoder = TransactionEncoder()
datos_codificados = encoder.fit_transform(grouped)
df = pd.DataFrame(datos_codificados, columns=encoder.columns_)

print(f"Número de transacciones: {df.shape[0]}\n")
df

Número de transacciones: 297453



,#2 Coffee Filters,#2 Cone White Coffee Filters,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,+Energy Black Cherry Vegetable & Fruit Juice,".5\"" Waterproof Tape",0 Calorie Acai Raspberry Water Beverage,0 Calorie Fuji Apple Pear Water Beverage,0 Calorie Strawberry Dragonfruit Water Beverage,0% Fat Black Cherry Greek Yogurt y,...,with Sweet Cinnamon Bunches Cereal,with Twist Ties Sandwich & Storage Bags,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297448,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
297449,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
297450,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
297451,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**¿Qué soporte debemos establecer para calcular los itemsets frecuentes?**

In [30]:
num_trans = df.shape[0]
min_support = 300 / num_trans
print(f"Soporte mínimo (300/{num_trans}): {min_support:.4f}\n")

Soporte mínimo (300/297453): 0.0010



**¿Cuántos itemsets frecuentes encontramos?**

In [ ]:
frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True, verbose = 1)
print(f"Número total de itemsets frecuentes: {len(frequent_itemsets)}")
frequent_itemsets

**¿Cuál de ellos tiene el soporte más alto?**

In [23]:
mayor_soporte = frequent_itemsets.loc[frequent_itemsets['support'].idxmax()]
print(f"Itemset con mayor soporte ({mayor_soporte['support']:.4f}): {mayor_soporte['itemsets']}\n")

Itemset con mayor soporte (0.1491): frozenset({'Banana'})



**¿Cuántos de los itemsets frecuentes contienen el producto "Seedless Red Grapes"?**

In [24]:
contains = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda s: 'Seedless Red Grapes' in s)]
print(f"Itemsets frecuentes que contienen 'Seedless Red Grapes': {len(contains)}")

Itemsets frecuentes que contienen 'Seedless Red Grapes': 1


**¿Cuál de los que tienen más de dos items tiene el soporte más alto?**

In [25]:
gt2 = contains[contains['itemsets'].apply(lambda s: len(s) > 2)]
if not gt2.empty:
    best_gt2 = gt2.loc[gt2['support'].idxmax()]
    print(f"De longitud mayor que 2, el que mayor soporte ({best_gt2['support']:.4f}) es {best_gt2['itemsets']}")
else:
    print("No hay itemsets de longitud mayor que 2 que contengan 'Seedless Red Grapes'.")

No hay itemsets de longitud mayor que 2 que contengan 'Seedless Red Grapes'.


**¿Cuántos de los itemset frecuentes son subconjuntos de otro itemset?**

In [26]:
all_sets = list(frequent_itemsets['itemsets'])
count_sub = sum(1 for s in all_sets if any((s < i) for i in all_sets if s != i))
print(f"Número de itemsets frecuentes que son subconjuntos de otro: {count_sub}\n")

Número de itemsets frecuentes que son subconjuntos de otro: 11



**Determina el valor necesario de confianza para que se generen entre 150 y 200 reglas con el soporte anteriormente definido.**

In [27]:
rules_all = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.0)


valores_confianza = [i/10 for i in range(11)]
confi_optima = None

for confi in valores_confianza:
    num = (rules_all['confidence'] >= confi).sum()
    print(f"Confianza: {confi:.2f}: Número de reglas: {num}")
    if 150 <= num <= 200:
        confi_optima = confi
        print(f"\nValor óptimo encontrado: {confi_optima:.2f}: {num} reglas")
        break

if not confi_optima:
    print("No hay confianza que genere entre 150 y 200 reglas con dicho soporte")

else: 
    rules = rules_all[rules_all['confidence'] >= confi_optima]

Confianza: 0.00: Número de reglas: 30
Confianza: 0.10: Número de reglas: 25
Confianza: 0.20: Número de reglas: 11
Confianza: 0.30: Número de reglas: 3
Confianza: 0.40: Número de reglas: 0
Confianza: 0.50: Número de reglas: 0
Confianza: 0.60: Número de reglas: 0
Confianza: 0.70: Número de reglas: 0
Confianza: 0.80: Número de reglas: 0
Confianza: 0.90: Número de reglas: 0
Confianza: 1.00: Número de reglas: 0
No hay confianza que genere entre 150 y 200 reglas con dicho soporte


**Indica el número de transacciones en las que se verifica la regla con la confianza más alta y que, además, incluyan “Organic Strawberries” en el antecedente o el consecuente**

In [28]:
if not confi_optima:
    print("No hay confianza que genere entre 150 y 200 reglas con dicho soporte")

else: 
    mask = (rules['antecedents'].map(lambda s: 'Organic Strawberries' in s) | 
            rules['consequents'].map(lambda s: 'Organic Strawberries' in s))

    if mask.sum() == 0:
        print("No hay reglas que incluyan 'Organic Strawberries'.")

    else:
        best_ogstraw = rules.loc[mask].nlargest(1, 'confidence').iloc[0]
        num_trans_ogstraw = int(best_ogstraw.support * num_trans)

        print("Regla con mayor confianza que incluye 'Organic Strawberries':")
        print(f"  {set(best_ogstraw['antecedents'])} → {set(best_ogstraw['consequents'])}")
        print(f"  Confianza: {best_ogstraw['confidence']:.4f}")
        print(f"  Nº transacciones que verifican la regla: {num_trans_ogstraw}")


No hay confianza que genere entre 150 y 200 reglas con dicho soporte
